<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Algoritmo%20cu%C3%A1ntico%20variacional%20para%20el%20c%C3%A1lculo%20de%20autovalores%20(VQE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

# **Algoritmo Cuántico Variacional para el cálculo de autovalores (VQE)**

### Contenidos
1. [Principio Variacional](#pv)

VQE es una generalización de QAOA para aproximar el estado base de un hamiltoniano H genérico.

Se suele usar en problemas de química computacional, por ejemplo, para obtener el mínimo estado de energía de una determinada molécula. También se puede usar en problemas de optimización combinatoria.

Se basa en el principio variacional, que establece que el mínimo valor esperado de un observable se alcanza siempre en un autovector del mismo. Es decir, que para cualquier estado arbitrario, el valor esperado de un observable es siempre mayor o igual al mínimo.


<a name="pv"></a>
## Principio variacional
  
El principio variacional permite estimar un límite superior para la energía del estado base (_ground state_) de un sistema cuántico.
    
Supongamos un estado cuántico, caracterizado por un hamiltoniano $H$, que representa la energía del sistema. Los posibles valores de energía son los autovalores $\lambda_i$ de la matriz $H$, y la energía del sistema en el estado $\vert u_i\rangle$ viene dada por el valor esperado de $H$ en ese estado:

$$
E(\vert u_i \rangle)\equiv \lambda_i = \langle H\rangle_{|u_i\rangle} = \langle u_i \vert H \vert u_i \rangle
$$

donde $|u_i\rangle$ es el autovector o autoestado asociado a $\lambda_i$.

El estado base del sistema es el correspondiente al mínimo de energía. Supongamos que ese mínimo corresponde al autoestado $|u_0\rangle$ con autovalor $\lambda_0$. Para cualquier autovalor $\lambda_i$ se tiene que:

\begin{align*}
    E(|u_0\rangle) \equiv \lambda_0 \le \lambda_i \equiv \langle H \rangle_{|u_i\rangle} = \langle u_i |H|u_i \rangle
\end{align*}  


Sea un estado arbitrario $|\psi\rangle$. Escribiendo $H$ en la base de sus autoestados $H = \sum_{i=1}^N \lambda_i |u_i\rangle\langle u_i|$ tenemos que el valor esperado de la energía en el estado $|\psi\rangle$:

\begin{align}
E(|\psi\rangle)=\langle H\rangle_{|\psi\rangle} = \langle \psi |H|\psi \rangle & = \
 \langle \psi | \left( \sum_{i=1}^N \lambda_i |u_i\rangle\langle u_i| \right) |\psi \rangle\\
 &= \sum_{i=1}^N \lambda_i \langle \psi |u_i\rangle\langle u_i|\psi \rangle\\
 &= \sum_{i=1}^N \lambda_i |\langle u_i|\psi \rangle|^2
\end{align}

Es decir, el valor esperado de $H$ en un estado cualquiera es una combinación lineal en la que los autovalores actúan como pesos. De esta expresión es facil ver que:

\begin{align}
    \langle H \rangle_{|\psi\rangle} = \langle \psi | H | \psi \rangle = \sum_{i = 1}^{N} \lambda_i | \langle u_i | \psi\rangle |^2 \ge  \sum_{i = 1}^{N} \lambda_0 | \langle u_i | \psi\rangle |^2 = \lambda_0 \sum_{i = 1}^{N} | \langle u_i | \psi\rangle |^2 = \lambda_0
\end{align}

Esa expresión se denomina _principio variacional_ y simplemente indica que el valor mínimo de energía es menor o igual que el valor esperado de $H$ en un estado arbitrario.



VQE aproxima el estado base de un hamiltoniano H. Para ello se crea un circuito cuántico variacional (denominado _forma variacional_ o _ansatz_) en un estado inicial $|\psi\rangle$. Ejecutando el circuito, se obtiene el valor esperado $\langle H \rangle_{|\psi\rangle}$. Se usa un  optimizador clásico para ajustar los parámetros del circuito con con vistas a encontrar los parámetros que minimicen $\langle H \rangle_{|\psi\rangle}$.

<center><img src="https://drive.google.com/uc?export=view&id=1O9RXUnTfu09PLD7O4XT1FrC2GcSE008t" alt="Circuito variacional" width="900"  /></center>


La idea de VQE es, partiendo del estado inicial, ir recorriendo el espacio de estados (o lo que es lo mismo, la esfera de Bloch) y calcular el valor esperado en $H$ en cada estado.

Por ejemplo, se podría elegir como valor inicial el estado $|0\rangle$ e ir aplicando rotaciones $R_Y$ o $R_x$:

<center><img src="https://drive.google.com/uc?export=view&id=1y94qu2xOloSXoTe-8t9nwgehEvgfZHG1" alt="VQE" width="200"  /></center>
(Fuente: <a href=https://www.mustythoughts.com/variational-quantum-eigensolver-explained>https://www.mustythoughts.com/variational-quantum-eigensolver-explained</a>)

En general, se usan ansätze $V(\theta)$ más complejos que permitan recorrer la esfera de Bloch.

#### Optimización de los parámetros

El algoritmo VQE parte de un Hamiltoniano $H$, cuyo estado base resuelve el problema, y procede como sigue:

1. Se elige un estado inicial (normalmente $|\psi(\theta)\rangle = |0\rangle$) y un _ansatz_ $V(\theta)$
2. Se le dan valores iniciales a los parámetros $\theta$
2. En la QPU se ejecuta el ansatz con esos parámetros para obtener un estado $|\psi(\theta)\rangle = V(\theta)|0\rangle$
3. Se mide el valor esperado del Hamiltoniano en ese estado $\langle\psi(\theta)|H|\psi(\theta)\rangle$
4. En la CPU se usa un algoritmo de optimización para modificar los parámetros del ansatz
    - Objetivo: reducir el valor esperado del Hamiltoniano
5. Se vuelve al paso 3 hasta que se alcanza un mínimo

Igual que en QAOA, el proceso de optimización de los parámetros puede ser un simple gradiente descendente, en el que cada parámetro se actualiza en la dirección que conduzca al mayor descenso de la energía, u otros método de optimización más sofisticados.

#### Obtención del valor esperado

A diferencia del QAOA, en el que el Hamiltoniano es una matriz diagonal, en VQE es más general $\implies$ no es trivial obtener el valor esperado.

Solución: Expresar el Hamiltoniano como suma de productos tensor de matrices de Pauli y obtener el valor esperado tomando medidas en diferentes bases.

#### Ejemplos de ansätze

La elección de la forma variacional o ansatz para VQE depende del problema a tratar. Un ejemplo es _RealAmplitudes_ que vimos antes. Otra forma es la _EfficientSU2_.

In [ ]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement='linear', insert_barriers=True)

ansatz.decompose().draw('mpl')

### Obtención de estados excitados

El algoritmo VQD (_Variational Quantum Deflaction_) es una extensión del VQE que permite obtener _estados excitados_, es decir, autoestados con mayor energía.

Para ello, si $|\psi_0\rangle$ es el estado base, es posible demostrar que el estado base del siguiente Hamiltoniano:

$$
H' = H + C|\psi_0\rangle\langle\psi_0|
$$
es el primer estado excitado de H

#### Referencias:

  - Peruzzo, et al. (2014). A variational eigenvalue solver on a photonic quantum processor. Nature communications, 5(1), 1-7. [arXiv:1304.3061](https://arxiv.org/abs/1304.3061)
  - Higgott, O., Wang, D., & Brierley, S. (2019). Variational quantum computation of excited states. Quantum, 3, 156. [arXiv:1805.08138](https://arxiv.org/abs/1805.08138)
  - Combarro, E.F. & Gonzákez-Castillo, S. (2023). A Practical Guide  to Quantum Machine Learning and Quantum Optimization, capítulo  7. Packt.
  - Qiskit tutorial: Variational quantum eigensolver https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver




---



---



---

